In [1]:
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import cv2 as cv
from typing import Optional

**Convolutional Neural Networks (CNN's)**
---

To process the non-linear patterns that images normally contain, we must break down the Network into the following layers:
  
  - Convolution Layer
  - Activation
  - Pooling

Each of the following makes up a block inside a convolutional network. From there we add an output transformation head containing nn.Linear and shape the transformation head according to the desired learning task. A Convolutional Neural Network can use multiple blocks of the layers above, and we generally use these models for image classification or time series prediction.
This notebook onlyfocus on 2D convolutions.

**Convolutional Layer**
---

As seen in the gif below the convolution builds a condensed representation of the input by calculating the dot product between a set of learnable parameters $\Theta$ (kernel) and the a windowed view of the input using a $NxN$ window matrix. We can then iteratively scale this latent representation to a more compute friendly size, while still retaining most of the information of the original image. As we can see in the example below, we use a kernel size of 3 : (a $3x3$ matrix of learnable parameters) and we transform the image from a $5x5$ image to a $3x3$ image. We now have 3 controllable parameters for our CNN, each of which will affect how much of the original image is represented in the layer output.

  - Stride: The number of steps the kernel takes between representations. (S)
  - N: Kernel dim (N)
  - Padding: image padding (P)

<img src="./resources/convolution.gif" width="585" height="425"/>

The size of the output of a convolutional layer depends on the following formula. Consider we have an image of $(WxWxD)$. The output from a convolutional layer of N: n, Padding: p, and Stride: s would be described by:

  - $W_{out} = \frac{W - N + 2P}{S} + 1$

One final thing to note about the convolution layer is; the nature of a convolution is equivariant to translation. This is to say, the representation of some image X and some image Y = X translated to the right by 5, will be more or less the same representation.

**Activation Layer**
---

Following the motivation behind activation functions in standard Dense Fully Connected Linear Layers, we use a Differentiable Non-Linear activation function to allow the model to output Non-Linear values. For this we can use ReLu, Sigmoid, tanh, etc.

**Pooling Layer**
---

Once we have our dense non-linear latent representation we combine and pool the learned representation using some combination function to reduce the dimension. Some examples of pooling layers can be found below:

  - Max Pooling: For an $NxN$ kernel matrix $\lambda$ : $MaxPool(X_{wi, hi}) = max(X_{wi, hi})$
  - Avg Pooling: For an $NxN$ kernel matrix $\lambda$ : $AvgPool(X_{wi, hi}) = \frac{1}{N^2} \sum(X_{wi, hi})$

To measure the size of the output after the pooling layer, we follow the following formula:

  - $W_{out} = \frac{W - N}{S} + 1$

**A PyTorch example**
---

In [ ]:
# Image model
class LeNet(nn.Module):
    def __init__(self, seed: Optional[float] = 0.0):
        super(LeNet, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square conv kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5x5 image dimension
        self.fc2 = nn.Linear(120, 84) # Classification output transformation head
        self.fc3 = nn.Linear(84, 10)

        # seed
        T.manual_seed(seed)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, int(x.nelement() / x.shape[0]))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x